In [25]:
%load_ext autoreload
%autoreload 2
import torch
import numpy as np

from tp_pt_vae import models

n_tracks = 5
n_measures = 4
measure_resolution = 48
n_beats = 4
n_pitches = 84
d_latent = 64

vae = models.VAE(
    n_tracks,
    n_measures,
    measure_resolution,
    n_beats,
    n_pitches,
    d_latent,
)

_sample = torch.FloatTensor(np.array([np.load("/home/ksawada/Documents/lab/lab_research/musegan/data/extracted/data/71843.npy").astype(np.float32), ]))
_sample = torch.permute(_sample, (0, 4, 1, 2, 3))

out = vae(_sample)
print(out[0][0].shape)




The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
torch.Size([1, 5, 4, 48, 84])


In [11]:
from torch.utils.data import DataLoader
from tqdm import tqdm


from tp_pt_vae.datasets import PianorollDataset
from tp_pt_vae.bin.train import Collater

n_tracks = 5
n_measures = 4
measure_resolution = 48
n_beats = 4
n_pitches = 84
d_latent = 64
batch_size = 1
num_workers = 1
pin_memory = True

dataset = PianorollDataset(
    pianoroll_list="/home/ksawada/Documents/lab/lab_research/tp_pt_vae/egs/lpd/data/lpd_train.txt",
    n_tracks=n_tracks,
    measure_resolution=measure_resolution,
    n_pitches=n_pitches,
    n_measures=n_measures,
    allow_cache=False,
)
collater = Collater()

dataloader = DataLoader(
    dataset=dataset,
    shuffle=True,
    collate_fn=collater,
    batch_size=batch_size,
    num_workers=num_workers,
    sampler=None,
    pin_memory=pin_memory,
)

total = len(dataset) * n_tracks * n_measures * measure_resolution * n_pitches
ones = 0
for i in tqdm(dataloader):
    ones += (i == 0).sum().item()

print(f"ones/total = {ones} / {total} = {ones/total}")

 98%|█████████▊| 90505/92140 [32:30<00:05, 276.89it/s]Traceback (most recent call last):
  File "/home/ksawada/Documents/lab/lab_research/tp_pt_vae/venv/lib/python3.9/site-packages/torch/utils/data/dataloader.py", line 1466, in __del__
  File "/home/ksawada/Documents/lab/lab_research/tp_pt_vae/venv/lib/python3.9/site-packages/torch/utils/data/dataloader.py", line 1449, in _shutdown_workers
  File "/home/ksawada/.pyenv/versions/3.9.1/lib/python3.9/multiprocessing/process.py", line 160, in is_alive
AssertionError: can only test a child process
100%|██████████| 92140/92140 [33:03<00:00, 46.44it/s] 

ones/total = 7289648693 / 7430169600 = 0.9810877928008535


In [12]:
1 - 0.9810877928008535

0.018912207199146547